In [ ]:
"""
Module: Google Earth Engine DEM Processing Script

Script Functionality:
This script initializes Google Earth Engine (GEE), defines a region of interest (ROI), 
loads and filters digital elevation model (DEM) images from the COPERNICUS/DEM/GLO30 dataset, 
and merges them into a single mosaic image. The resulting image is then exported to Google Drive 
for further analysis or visualization.

Input Parameters:
A shapefile containing the boundary for the region of interest (ROI) should be available in the specified GEE asset path.

Dependencies:
Earth Engine API (ee) : pip install earthengine-api
Geemap : pip install geemap (for easier GEE integration with Python)

Version Information:
Version: v1.0
Change Log: October 11 2024.

"""
import ee
import geemap

# Initialize GEE
ee.Authenticate()
ee.Initialize()

# Define the region of interest (ROI), loading the boundary from a shapefile
roi = ee.FeatureCollection('projects/ee-meiyang150269/assets/boundary')

# Load and filter images from the COPERNICUS/DEM/GLO30 dataset, limited to the ROI range
dem_collection = ee.ImageCollection('COPERNICUS/DEM/GLO30').filterBounds(roi)

# Print the number of images within the ROI
image_count = dem_collection.size().getInfo()
print(f"Number of images in the region of interest: {image_count}")

# Use mosaic to merge images and clip to ROI, ensuring it is stitched into a single image
dem_mosaic = dem_collection.mosaic().clip(roi.geometry())

# Standardize data type to Float32 to prevent type mismatch during export
dem_mosaic = dem_mosaic.toFloat()

# Export DEM Mosaic data to Google Drive, ensuring the area matches the ROI
export_dem_mosaic = ee.batch.Export.image.toDrive(
    image=dem_mosaic,
    description='dem_mosaic_export',
    folder='GEE_exports',
    scale=30,  # Resolution during export
    region=roi.geometry().bounds().getInfo()['coordinates'],  # Use the boundary of the ROI as the clipping area
    crs='EPSG:4326',
    maxPixels=1e10
)
export_dem_mosaic.start()

print("DEM Mosaic export task has started. Please check your Google Drive.")

# End of the script

In [1]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn) 
Editor: Mei Yang (1024072120@njupt.edu.cn)
Date: October 11, 2024 
This code is protected by copyright and may not be copied or distributed without permission.

Module: Raster Merging Script

Script Functionality:
This script merges multiple raster (.tif) files from a specified input directory into a single 
GeoTIFF file using the GDAL library. It ensures lossless merging of raster data, which is commonly 
used in geospatial analysis for creating larger datasets from individual tiles.

Input Parameters:
The script requires a directory of raster files in .tif format for merging.

Dependencies:
GDAL (Geospatial Data Abstraction Library) : pip install gdal


Version Information:
Version: v1.0
Change Log: October 11 2024.
"""

import os
from osgeo import gdal

# Function: Lossless merging of raster data
def merge_rasters(input_dir, output_file):
    # Get all .tif files in the directory
    raster_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.tif')]

    # Use gdal.Warp for lossless merging
    gdal.Warp(output_file, raster_files, format='GTiff', options=["COMPRESS=NONE"])

    print(f"Merging completed, output file: {output_file}")

# Example execution
input_dir = r'D:\ghana\DEM'  # Path to the folder with tile data
output_file = r'D:\ghana\DEM\output_merged.tif'  # Path for the output file

merge_rasters(input_dir, output_file)

# End of the script

合并完成，输出文件：D:\ghana\DEM\output_merged.tif


ruggedness calculation

In [2]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn) 
Editor: Mei Yang (1024072120@njupt.edu.cn)
Date: October 11, 2024 
This code is protected by copyright and may not be copied or distributed without permission.

Module: DEM Smoothing and Terrain Ruggedness Calculation

Script Functionality:
This script processes a digital elevation model (DEM) by applying a Gaussian filter for smoothing 
and calculating the terrain ruggedness index (TRI). The smoothing is performed in blocks (tiles) 
to handle large datasets efficiently, and the ruggedness is computed using GDAL's DEMProcessing 
function. Both smoothed and ruggedness results are saved as GeoTIFF files.

Input Parameters:
A DEM raster file in .tif format is required as input for both smoothing and ruggedness calculation.

Dependencies:
NumPy (for numerical operations) : pip install numpy
SciPy (for applying Gaussian filters) : pip install scipy 
GDAL (for terrain ruggedness calculation) : pip install gdal 
Rasterio (for reading and writing raster files) : pip install rasterio

Version Information:
Version: v1.0
Change Log: October 11 2024.
"""
import os
import numpy as np
import scipy.ndimage as ndimage
from osgeo import gdal
import rasterio
from rasterio.windows import Window

# Function: Apply Gaussian filter to smooth DEM data (block processing)
def smooth_dem(input_raster, output_raster, sigma=1, block_size=512):
    with rasterio.open(input_raster) as src:
        profile = src.profile  # Get the file's metadata

        # Create an output file with the same profile
        profile.update(dtype=rasterio.float32)
        with rasterio.open(output_raster, 'w', **profile) as dst:
            # Process the raster in blocks (tiles)
            for ji, window in src.block_windows(1):  # Iterate through raster by block
                dem_data = src.read(1, window=window)  # Read the data block

                # Apply Gaussian filter for smoothing to the block
                smoothed_block = ndimage.gaussian_filter(dem_data, sigma=sigma)

                # Write the smoothed block to the output raster
                dst.write(smoothed_block.astype(np.float32), 1, window=window)

# Function: Calculate terrain ruggedness
def calculate_ruggedness(input_raster, output_ruggedness_raster):
    gdal.DEMProcessing(output_ruggedness_raster, input_raster, 'TRI', format='GTiff')

# Example run - Process single DEM file
input_dem = r'D:\ghana\DEM\clipped_DEM\output_merged.tif'  # Input DEM file
smoothed_dem = r'D:\ghana\DEM\clipped_DEM\slope_ruggedness\smoothed.tif'  # Smoothed DEM output file
ruggedness_dem = r'D:\ghana\DEM\clipped_DEM\slope_ruggedness\ruggedness.tif'  # Ruggedness output file

# 1. Smooth the DEM with Gaussian filter (block size = 512x512 pixels)
smooth_dem(input_dem, smoothed_dem, sigma=1, block_size=512)

# 2. Calculate ruggedness based on the smoothed DEM
calculate_ruggedness(smoothed_dem, ruggedness_dem)

print(f"Processed smoothing and ruggedness for: {input_dem}")

# End of the script

Processed smoothing and ruggedness for: D:\ghana\DEM\clipped_DEM\output_merged.tif


slope calculation

In [4]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn) 
Editor: Mei Yang (1024072120@njupt.edu.cn)
Date: October 11, 2024 
This code is protected by copyright and may not be copied or distributed without permission.

Module: DEM Smoothing and Slope Calculation in Chunks

Script Functionality:
This script processes a digital elevation model (DEM) by applying a Gaussian filter for smoothing 
and calculates the slope of the terrain using numpy's gradient function. The DEM is processed in 
chunks to efficiently handle large datasets, allowing for better memory management. Both the 
smoothed DEM and slope are saved as GeoTIFF files.

Input Parameters:
A DEM raster file in .tif format is required as input.

Dependencies:
NumPy (for numerical operations) : pip install numpy 
SciPy (for applying Gaussian filters) : pip install scipy 
Rasterio (for reading and writing raster files, and processing in chunks) : pip install rasterio

Version Information:
Version: v1.0
Change Log: October 11 2024.
"""

import numpy as np
import scipy.ndimage as ndimage
import rasterio
from rasterio.windows import Window
from rasterio.enums import Resampling

# File paths
input_dem = r"D:\ghana\DEM\clipped_DEM\output_merged.tif"
output_smoothed_dem = r"D:\ghana\DEM\clipped_DEM\slope_ruggedness\smoothed_dem.tif"
output_slope = r"D:\ghana\DEM\clipped_DEM\slope_ruggedness\slope_smoothed.tif"

# Chunk size
chunk_size = 1024  # You can adjust the chunk size based on available memory

# Read DEM metadata
with rasterio.open(input_dem) as src:
    transform = src.transform
    crs = src.crs
    profile = src.profile
    width = src.width
    height = src.height

# 1. Process the DEM in chunks (smoothing and slope calculation)
with rasterio.open(input_dem) as src:
    # Update the profile for the output (for smoothed DEM and slope)
    profile.update(dtype=rasterio.float32)

    # Prepare output files for smoothed DEM and slope
    with rasterio.open(output_smoothed_dem, 'w', **profile) as smoothed_dst, \
         rasterio.open(output_slope, 'w', **profile) as slope_dst:

        # Process the DEM in chunks
        for i in range(0, height, chunk_size):
            for j in range(0, width, chunk_size):
                # Define the window
                window = Window(j, i, min(chunk_size, width - j), min(chunk_size, height - i))

                # Read the chunk of DEM data
                dem_chunk = src.read(1, window=window)

                # Apply Gaussian filter to smooth the DEM
                smoothed_chunk = ndimage.gaussian_filter(dem_chunk, sigma=1)

                # Calculate the slope
                x, y = np.gradient(smoothed_chunk)
                slope_chunk = np.sqrt(x**2 + y**2)

                # Write the smoothed DEM chunk to the output file
                smoothed_dst.write(smoothed_chunk.astype(np.float32), 1, window=window)

                # Write the slope chunk to the output file
                slope_dst.write(slope_chunk.astype(np.float32), 1, window=window)

print("DEM Smoothing and Slope Calculation Complete.")

# End of the script

DEM Smoothing and Slope Calculation Complete.


In [6]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn) 
Editor: Mei Yang (1024072120@njupt.edu.cn)
Date: October 11, 2024 
This code is protected by copyright and may not be copied or distributed without permission.

Module: DEM Resampling and Reprojection Script

Script Functionality:
This script resamples and reprojects a set of DEM (Digital Elevation Model) raster files using GDAL. 
The output files are resampled to match the projection and resolution of a reference file. It iterates 
through the input folder, processes each `.tif` file, and saves the resampled output to the specified 
output folder.

Input Parameters:
A reference DEM raster file in `.tif` format that defines the target resolution and projection.
A folder containing the input raster files that need to be resampled and reprojected.

Dependencies:
GDAL (for reading, resampling, and reprojection) : pip install gdal 
Rasterio (for additional raster processing, although not used heavily in this script) : pip install rasterio 
NumPy (for numerical operations if needed, though not directly used in this version) : pip install numpy

Version Information:
Version: v1.0
Change Log: October 11 2024.
"""
from osgeo import gdal
import rasterio
from rasterio.enums import Resampling
import os
import numpy as np

referencefilepath = r'D:\ghana\0.01.tif'
inputfolderpath = r'D:\ghana\DEM\slope_ruggedness'
inputfilepaths = []
outputfolderpath = r'D:\ghana\DEM\0.01'
outputfilepaths = []

# Get projection information
referencefile = gdal.Open(referencefilepath, gdal.GA_ReadOnly)
rfproj = referencefile.GetProjection()
bandreferencefile = referencefile.GetRasterBand(1)
rfTrans = referencefile.GetGeoTransform()
rfwidth = referencefile.RasterXSize
rfheight = referencefile.RasterYSize
nbands = referencefile.RasterCount

# Read data
file_list = os.listdir(inputfolderpath)
for filename in file_list:
    if filename.endswith('.tif'):
        inputpath = os.path.join(inputfolderpath, filename)
        outputpath = os.path.join(outputfolderpath, '0.005_' + filename)
        inputfilepaths.append(inputpath)
        outputfilepaths.append(outputpath)

print(inputfilepaths)

# Resampling
def ReprojectImages(inputFilePaths, outputFilePaths):
    for i in range(len(inputFilePaths)):
        inputfilePath = inputFilePaths[i]
        outputfilePath = outputFilePaths[i]
        # Get input raster file information
        inputrasfile = gdal.Open(inputfilePath, gdal.GA_ReadOnly)
        inputProj = inputrasfile.GetProjection()
        # Create resampled output file (set projection and six parameters)
        driver = gdal.GetDriverByName('GTiff')
        output = driver.Create(outputfilePath, rfwidth, rfheight, nbands, bandreferencefile.DataType)
        output.SetGeoTransform(rfTrans)
        output.SetProjection(rfproj)
        # Parameters: input dataset, output file, input projection, reference projection, resampling method (Nearest Neighbor, Bilinear, Cubic Convolution, etc.), callback function
        gdal.ReprojectImage(inputrasfile, output, inputProj, rfproj, gdal.GRA_NearestNeighbour)

ReprojectImages(inputfilepaths, outputfilepaths)

# End of the script

['D:\\ghana\\DEM\\slope_ruggedness\\DEM.tif', 'D:\\ghana\\DEM\\slope_ruggedness\\ruggedness.tif', 'D:\\ghana\\DEM\\slope_ruggedness\\slope.tif']


The proportion of areas with a slope of 5 degrees or less

In [1]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn) 
Editor: Mei Yang (1024072120@njupt.edu.cn)
Date: October 11, 2024 
This code is protected by copyright and may not be copied or distributed without permission.

Module: Slope Percentage Calculation Script

Script Functionality:
This script calculates the percentage of areas with slope less than or equal to 5 degrees (or a specified threshold) 
based on a high-resolution slope raster (0.00083333333-degree resolution). It processes the data in chunks to avoid 
memory errors and outputs the result to a new raster file that retains the resolution of the original raster.

The proportion of pixels with slope below the threshold is calculated for each corresponding low-resolution template 
pixel (0.005-degree resolution) and then written back to the original high-resolution raster format.

Input Parameters:
A high-resolution slope raster in .tif format (e.g., 0.00083333333-degree resolution).
A low-resolution template raster in .tif format (e.g., 0.005-degree resolution).
Output file path for the result.

Dependencies:
GDAL (for raster reading and writing) : pip install gdal 
NumPy (for numerical operations and array processing) : pip install numpy

Version Information:
Version: v1.0
Change Log: October 11 2024.
"""
import os
import numpy as np
from osgeo import gdal

def calculate_slope_percentage(input_tif, template_tif, output_tif, slope_threshold=5, chunk_size=1024):
    """
    Calculate the percentage of areas with slope less than or equal to 5 degrees
    at a 0.005-degree raster resolution, and write the result back to the original high-resolution raster.
    
    :param input_tif: Input 0.00083333333-degree slope TIF file
    :param template_tif: 0.005-degree template TIF file
    :param output_tif: Output TIF file for the percentage result
    :param slope_threshold: Slope threshold, default is 5 degrees
    :param chunk_size: Size of chunks to process in memory to avoid MemoryError
    """
    # Open the input high-resolution slope raster file
    src_ds = gdal.Open(input_tif)
    if src_ds is None:
        print(f"Cannot open input TIF file: {input_tif}")
        return
    src_band = src_ds.GetRasterBand(1)
    
    # Open the template low-resolution raster file
    template_ds = gdal.Open(template_tif)
    if template_ds is None:
        print(f"Cannot open template TIF file: {template_tif}")
        return
    
    # Get geographic information of input and template
    src_transform = src_ds.GetGeoTransform()
    template_transform = template_ds.GetGeoTransform()

    # Get resolution of input and template
    src_pixel_width = src_transform[1]  # Pixel size of high-resolution raster (0.00083333333 degrees)
    template_pixel_width = template_transform[1]  # Pixel size of low-resolution raster (0.005 degrees)

    # Calculate the number of high-resolution pixels contained in each low-resolution template pixel
    ratio = int(template_pixel_width / src_pixel_width)

    # Get the size of the original high-resolution raster
    src_x_size = src_ds.RasterXSize
    src_y_size = src_ds.RasterYSize

    # Create output TIF file, maintaining the size of high-resolution raster
    driver = gdal.GetDriverByName("GTiff")
    out_ds = driver.Create(output_tif, src_x_size, src_y_size, 1, gdal.GDT_Float32)
    out_ds.SetGeoTransform(src_transform)  # Use geographic information of original high-resolution raster
    out_ds.SetProjection(src_ds.GetProjection())
    out_band = out_ds.GetRasterBand(1)

    # Process the DEM in chunks to avoid MemoryError
    for i in range(0, src_y_size, chunk_size):
        for j in range(0, src_x_size, chunk_size):
            # Define the chunk window for reading and processing
            x_chunk_size = min(chunk_size, src_x_size - j)
            y_chunk_size = min(chunk_size, src_y_size - i)

            # Read a chunk of the input high-resolution slope raster
            src_array = src_band.ReadAsArray(j, i, x_chunk_size, y_chunk_size)

            # Initialize output array for this chunk
            out_chunk_array = np.zeros((y_chunk_size, x_chunk_size), dtype=np.float32)

            # Process each sub-region within the chunk
            for y in range(0, y_chunk_size, ratio):
                for x in range(0, x_chunk_size, ratio):
                    # Get the area of the high-resolution raster within the template pixel range
                    sub_array = src_array[y:y+ratio, x:x+ratio]

                    # Count the number of pixels within the area with slope less than or equal to the threshold
                    target_count = np.sum(sub_array <= slope_threshold)
                    total_count = sub_array.size

                    # Calculate proportion
                    proportion = target_count / total_count if total_count > 0 else 0

                    # Assign the proportion to all pixels in the original high-resolution raster
                    out_chunk_array[y:y+ratio, x:x+ratio] = proportion

            # Write the chunk to the output TIF file
            out_band.WriteArray(out_chunk_array, j, i)

    out_band.FlushCache()
    print(f"Percentage of areas with slope less than or equal to {slope_threshold} degrees has been saved to: {output_tif}")

    # Close datasets
    src_ds = None
    template_ds = None
    out_ds = None

# Example usage
input_tif = r"D:\ghana\DEM\slope_ruggedness\slope.tif"  # 0.00083333333-degree slope raster
template_tif = r"D:\ghana\0.005.tif"  # 0.005-degree template raster
output_tif = r"D:\ghana\DEM\slope_ruggedness\slope_proportion_0.005.tif"  # Output TIF file path

calculate_slope_percentage(input_tif, template_tif, output_tif)

# End of the script

d:\ProgramFiles\anaconda3\envs\gee\Lib\site-packages\osgeo\gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


Percentage of areas with slope less than or equal to 5 degrees has been saved to: D:\ghana\DEM\slope_ruggedness\slope_proportion_0.005.tif


The proportion of areas with a slope of 10 degrees or less

In [2]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn) 
Editor: Mei Yang (1024072120@njupt.edu.cn)
Date: October 11, 2024 
This code is protected by copyright and may not be copied or distributed without permission.

Module: Slope Percentage Calculation Script

Script Functionality:
This script calculates the percentage of areas with a slope less than or equal to a specified threshold 
(slope_threshold, default is 5 degrees) based on a high-resolution slope raster (0.00083333333 degrees). 
It resamples the slope information to match a lower resolution template raster (0.01-degree resolution), 
and stores the result as a proportion in the original high-resolution raster. The output is saved as a new 
GeoTIFF file.

Input Parameters:
A high-resolution slope raster file in .tif format (e.g., 0.00083333333-degree resolution).
A low-resolution template raster file (e.g., 0.01-degree resolution).

Dependencies:
NumPy (for numerical array operations) : pip install numpy
GDAL (for reading and writing GeoTIFF raster files) : pip install gdal

Version Information:
Version: v1.0
Change Log: October 11 2024.
"""
import os
import numpy as np
from osgeo import gdal

def calculate_slope_percentage(input_tif, template_tif, output_tif, slope_threshold=5, chunk_size=1024):
    """
    Calculate the percentage of areas with a slope less than or equal to 5 degrees
    at a 0.01-degree raster resolution, and write the result back to the original high-resolution raster.
    
    :param input_tif: Input 0.00083333333-degree slope TIF file
    :param template_tif: 0.01-degree template TIF file
    :param output_tif: Output TIF file for the percentage result
    :param slope_threshold: Slope threshold, default is 5 degrees
    :param chunk_size: Size of chunks to process in memory to avoid MemoryError
    """
    # Open the input high-resolution slope raster file
    src_ds = gdal.Open(input_tif)
    if src_ds is None:
        print(f"Cannot open input TIF file: {input_tif}")
        return
    src_band = src_ds.GetRasterBand(1)
    
    # Open the template low-resolution raster file
    template_ds = gdal.Open(template_tif)
    if template_ds is None:
        print(f"Cannot open template TIF file: {template_tif}")
        return
    
    # Get geographic information of input and template
    src_transform = src_ds.GetGeoTransform()
    template_transform = template_ds.GetGeoTransform()

    # Get resolution of input and template
    src_pixel_width = src_transform[1]  # Pixel size of high-resolution raster (0.00083333333 degrees)
    template_pixel_width = template_transform[1]  # Pixel size of low-resolution raster (0.01 degrees)

    # Calculate the number of high-resolution pixels contained in each low-resolution template pixel
    ratio = int(template_pixel_width / src_pixel_width)

    # Get the size of the original high-resolution raster
    src_x_size = src_ds.RasterXSize
    src_y_size = src_ds.RasterYSize

    # Create output TIF file, maintaining the size of high-resolution raster
    driver = gdal.GetDriverByName("GTiff")
    out_ds = driver.Create(output_tif, src_x_size, src_y_size, 1, gdal.GDT_Float32)
    out_ds.SetGeoTransform(src_transform)  # Use geographic information of original high-resolution raster
    out_ds.SetProjection(src_ds.GetProjection())
    out_band = out_ds.GetRasterBand(1)

    # Process the DEM in chunks to avoid MemoryError
    for i in range(0, src_y_size, chunk_size):
        for j in range(0, src_x_size, chunk_size):
            # Define the chunk window for reading and processing
            x_chunk_size = min(chunk_size, src_x_size - j)
            y_chunk_size = min(chunk_size, src_y_size - i)

            # Read a chunk of the input high-resolution slope raster
            src_array = src_band.ReadAsArray(j, i, x_chunk_size, y_chunk_size)

            # Initialize output array for this chunk
            out_chunk_array = np.zeros((y_chunk_size, x_chunk_size), dtype=np.float32)

            # Process each sub-region within the chunk
            for y in range(0, y_chunk_size, ratio):
                for x in range(0, x_chunk_size, ratio):
                    # Get the area of the high-resolution raster within the template pixel range
                    sub_array = src_array[y:y+ratio, x:x+ratio]

                    # Count the number of pixels within the area with slope less than or equal to the threshold
                    target_count = np.sum(sub_array <= slope_threshold)
                    total_count = sub_array.size

                    # Calculate proportion
                    proportion = target_count / total_count if total_count > 0 else 0

                    # Assign the proportion to all pixels in the original high-resolution raster
                    out_chunk_array[y:y+ratio, x:x+ratio] = proportion

            # Write the chunk to the output TIF file
            out_band.WriteArray(out_chunk_array, j, i)

    out_band.FlushCache()
    print(f"Percentage of areas with slope less than or equal to {slope_threshold} degrees has been saved to: {output_tif}")

    # Close datasets
    src_ds = None
    template_ds = None
    out_ds = None

# Example usage
input_tif = r"D:\ghana\DEM\slope_ruggedness\slope.tif"  # 0.00083333333-degree slope raster
template_tif = r"D:\ghana\0.01.tif"  # 0.01-degree template raster
output_tif = r"D:\ghana\DEM\slope_ruggedness\slope_proportion_0.01.tif"  # Output TIF file path

calculate_slope_percentage(input_tif, template_tif, output_tif)

# End of the script

Percentage of areas with slope less than or equal to 5 degrees has been saved to: D:\ghana\DEM\slope_ruggedness\slope_proportion_0.01.tif


In [3]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn) 
Editor: Mei Yang (1024072120@njupt.edu.cn)
Date: November 3, 2024 
This code is protected by copyright and may not be copied or distributed without permission.

Module: Slope Percentage Calculation Script

Script Functionality:
This script calculates the percentage of areas with a slope less than or equal to a specified threshold 
(slope_threshold, default is 5 degrees) based on a high-resolution slope raster (0.00083333333 degrees). 
It aggregates the slope information to match a lower resolution of 0.05 degrees (or a specified resolution) 
and outputs the proportion of such areas as a new GeoTIFF file. This allows for efficient analysis of 
slope characteristics over larger areas by reducing the resolution while preserving essential data.

Input Parameters:
A high-resolution slope raster file in .tif format (e.g., 0.00083333333-degree resolution).
An optional low-resolution template raster file (e.g., 0.05-degree resolution).

Dependencies:
NumPy (for numerical array operations): pip install numpy
GDAL (for reading and writing GeoTIFF raster files): pip install gdal

Version Information:
Version: v1.0
Change Log: November 3, 2024.
"""

import os
import numpy as np
from osgeo import gdal

def calculate_slope_percentage(input_tif, output_tif, slope_threshold=5, chunk_size=1024):
    
    # Open the input high-resolution slope raster file
    src_ds = gdal.Open(input_tif)
    if src_ds is None:
        print(f"Cannot open input TIF file: {input_tif}")
        return
    src_band = src_ds.GetRasterBand(1)
    
    # Get geographic information of input
    src_transform = src_ds.GetGeoTransform()
    
    # Set template pixel width to 0.05 degrees
    template_pixel_width = 0.05
    src_pixel_width = src_transform[1]  # Pixel size of high-resolution raster (0.00083333333 degrees)

    # Calculate the number of high-resolution pixels contained in each low-resolution template pixel
    ratio = int(template_pixel_width / src_pixel_width)

    # Get the size of the original high-resolution raster
    src_x_size = src_ds.RasterXSize
    src_y_size = src_ds.RasterYSize

    # Create output TIF file, maintaining the size of high-resolution raster
    driver = gdal.GetDriverByName("GTiff")
    out_ds = driver.Create(output_tif, src_x_size, src_y_size, 1, gdal.GDT_Float32)
    out_ds.SetGeoTransform(src_transform)  # Use geographic information of original high-resolution raster
    out_ds.SetProjection(src_ds.GetProjection())
    out_band = out_ds.GetRasterBand(1)

    # Process the DEM in chunks to avoid MemoryError
    for i in range(0, src_y_size, chunk_size):
        for j in range(0, src_x_size, chunk_size):
            # Define the chunk window for reading and processing
            x_chunk_size = min(chunk_size, src_x_size - j)
            y_chunk_size = min(chunk_size, src_y_size - i)

            # Read a chunk of the input high-resolution slope raster
            src_array = src_band.ReadAsArray(j, i, x_chunk_size, y_chunk_size)

            # Initialize output array for this chunk
            out_chunk_array = np.zeros((y_chunk_size, x_chunk_size), dtype=np.float32)

            # Process each sub-region within the chunk
            for y in range(0, y_chunk_size, ratio):
                for x in range(0, x_chunk_size, ratio):
                    # Get the area of the high-resolution raster within the template pixel range
                    sub_array = src_array[y:y+ratio, x:x+ratio]

                    # Count the number of pixels within the area with slope less than or equal to the threshold
                    target_count = np.sum(sub_array <= slope_threshold)
                    total_count = sub_array.size

                    # Calculate proportion
                    proportion = target_count / total_count if total_count > 0 else 0

                    # Assign the proportion to all pixels in the original high-resolution raster
                    out_chunk_array[y:y+ratio, x:x+ratio] = proportion

            # Write the chunk to the output TIF file
            out_band.WriteArray(out_chunk_array, j, i)

    out_band.FlushCache()
    print(f"Percentage of areas with slope less than or equal to {slope_threshold} degrees has been saved to: {output_tif}")

    # Close datasets
    src_ds = None
    out_ds = None

# Example usage
input_tif = r"D:\ghana\DEM\slope_ruggedness\slope.tif"  # 0.00083333333-degree slope raster
output_tif = r"D:\ghana\DEM\slope_ruggedness\slope_proportion_0.05.tif"  # Output TIF file path

calculate_slope_percentage(input_tif, output_tif)

# End of the script

Percentage of areas with slope less than or equal to 5 degrees has been saved to: D:\ghana\DEM\slope_ruggedness\slope_proportion_0.05.tif


In [ ]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn)
Editor: Mei Yang (1024072120@njupt.edu.cn)
Date: November 03, 2024
This code is protected by copyright and may not be copied or distributed without permission.

Module: DEM Raster Resampling Script

Script Functionality:
This script resamples Digital Elevation Model (DEM) raster data files (.tif) located in a specified input folder. 
It processes each raster file to reduce its resolution by averaging values based on a defined scale factor (5), 
resulting in a new resolution of 0.05. The resampled raster files are saved in a designated output folder, 
with filenames updated to reflect the new resolution.

Input Parameters:
input_folder: A folder containing DEM raster files (.tif) to be resampled.
output_folder: A folder where the resampled DEM raster files will be saved.

Dependencies:
GDAL: pip install gdal (for raster processing and reprojection)
Rasterio: pip install rasterio (for raster file operations)
NumPy: pip install numpy (for numerical array operations)

Version Information:
Version: v1.0
Change Log: November 03, 2024.
"""

import os
import rasterio
from rasterio.enums import Resampling
import numpy as np

# Set input and output folder paths
input_folder = r"D:\ghana\11.3\DEM"  # Replace with your input folder path
output_folder = r"D:\ghana\11.3\DEM1"  # Replace with your output folder path

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process all TIFF files in batch
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):
        input_file = os.path.join(input_folder, filename)  # Construct full path
        print(f"Processing file: {input_file}")
        
        with rasterio.open(input_file) as src:
            data = src.read(1)  # Read the first band
            profile = src.profile
            
            # Set new resolution (0.05) and calculate scale factor
            scale_factor = 5  # Aggregation factor from 0.01 to 0.05
            new_height = data.shape[0] // scale_factor
            new_width = data.shape[1] // scale_factor

            # Mean aggregation resampling
            resampled_data = data[:new_height * scale_factor, :new_width * scale_factor].reshape(
                new_height, scale_factor, new_width, scale_factor).mean(axis=(1, 3))

            # Update profile
            profile.update({
                "height": new_height,
                "width": new_width,
                "transform": src.transform * src.transform.scale(scale_factor, scale_factor)
            })

        # Generate new output filename
        new_filename = filename.replace("0.01", "0.05")  # Replace "0.01" in the filename with "0.05"
        output_file = os.path.join(output_folder, new_filename)

        # Export the result
        with rasterio.open(output_file, "w", **profile) as dst:
            dst.write(resampled_data, 1)

        print(f"File saved to: {output_file}")

print("All files have been processed.")

# End of the script

处理文件: D:\ghana\11.3\DEM\0.01_DEM.tif
文件已保存到: D:\ghana\11.3\DEM1\0.05_DEM.tif
处理文件: D:\ghana\11.3\DEM\0.01_ruggedness.tif
文件已保存到: D:\ghana\11.3\DEM1\0.05_ruggedness.tif
处理文件: D:\ghana\11.3\DEM\0.01_slope.tif
文件已保存到: D:\ghana\11.3\DEM1\0.05_slope.tif
所有文件处理完成。


In [ ]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn) 
Editor: Mei Yang (1024072120@njupt.edu.cn)
Date: November 04, 2024
This code is protected by copyright and may not be copied or distributed without permission.

Module: Elevation Raster Resampling Script

Script Functionality:
This script resamples elevation raster data based on a reference raster file. 
It reads raster files from a specified input folder, resamples them to match the resolution and projection 
of the reference raster, and saves the resampled output files to a designated folder.

Input Parameters:
referencefilepath: A reference raster file (.tif) with the desired target resolution and projection.
inputfolderpath: A folder containing elevation raster files (.tif) that need to be resampled.

Dependencies:
GDAL : pip install gdal (for raster processing and reprojection)
Rasterio : pip install rasterio (optional, for additional raster operations)
NumPy : pip install numpy (for numerical array operations if needed)


Version Information:
Version: v1.0
Change Log: November 04, 2024
"""
from osgeo import gdal
import rasterio
from rasterio.enums import Resampling
import os
import numpy as np

referencefilepath = r'D:\ghana\0.005.tif'
inputfolderpath = r'D:\ghana\DEM\slope_ruggedness\slope_proportion_0.005'
inputfilepaths = []
outputfolderpath = r'D:\ghana\DEM\slope_ruggedness\slope_proportion_0.005'
outputfilepaths = []

# Get projection information
referencefile = gdal.Open(referencefilepath, gdal.GA_ReadOnly)
rfproj = referencefile.GetProjection()
bandreferencefile = referencefile.GetRasterBand(1)
rfTrans = referencefile.GetGeoTransform()
rfwidth = referencefile.RasterXSize
rfheight = referencefile.RasterYSize
nbands = referencefile.RasterCount

# Read data
file_list = os.listdir(inputfolderpath)
for filename in file_list:
    if filename.endswith('.tif'):
        inputpath = os.path.join(inputfolderpath, filename)
        outputpath = os.path.join(outputfolderpath, 'r' + filename)
        inputfilepaths.append(inputpath)
        outputfilepaths.append(outputpath)

print(inputfilepaths)

# Resampling
def ReprojectImages(inputFilePaths, outputFilePaths):
    for i in range(len(inputFilePaths)):
        inputfilePath = inputFilePaths[i]
        outputfilePath = outputFilePaths[i]
        # Get input raster file information
        inputrasfile = gdal.Open(inputfilePath, gdal.GA_ReadOnly)
        inputProj = inputrasfile.GetProjection()
        # Create resampled output file (set projection and six parameters)
        driver = gdal.GetDriverByName('GTiff')
        output = driver.Create(outputfilePath, rfwidth, rfheight, nbands, bandreferencefile.DataType)
        output.SetGeoTransform(rfTrans)
        output.SetProjection(rfproj)
        # Parameters: input dataset, output file, input projection, reference projection, resampling method (Nearest Neighbor, Bilinear, Cubic Convolution, etc.), callback function
        gdal.ReprojectImage(inputrasfile, output, inputProj, rfproj, gdal.GRA_NearestNeighbour)

ReprojectImages(inputfilepaths, outputfilepaths)

# End of the script

In [ ]:
"""
Copyright (c) 2024 Yan Jin (jinyan@njupt.edu.cn) and Yong Ge (gey@lreis.ac.cn)
Editor: Mei Yang (1024072120@njupt.edu.cn)
Date: November 04, 2024
This code is protected by copyright and may not be copied or distributed without permission.

Module: Elevation Raster Resampling Script

Script Functionality:
This script resamples elevation raster data files (.tif) located in a specified input folder. 
It processes each raster file to reduce its resolution by averaging values based on a defined scale factor (5), 
resulting in a new resolution of 0.05. The resampled raster files are saved in a designated output folder, 
with filenames updated to reflect the new resolution.

Input Parameters:
input_folder: A folder containing elevation raster files (.tif) to be resampled.
output_folder: A folder where the resampled elevation raster files will be saved.

Dependencies:
GDAL: pip install gdal (for raster processing and reprojection)
Rasterio: pip install rasterio (for raster file operations)
NumPy: pip install numpy (for numerical array operations)

Version Information:
Version: v1.0
Change Log: November 04, 2024.
"""

import os
import rasterio
from rasterio.enums import Resampling
import numpy as np

# Set input and output folder paths
input_folder = r"D:\ghana\DEM\slope_ruggedness\slope_proportion_0.01"  # Replace with your input folder path
output_folder = r"D:\ghana\DEM\slope_ruggedness\slope_proportion_0.05"  # Replace with your output folder path

# Ensure the output folder exists
os.makedirs(output_folder, exist_ok=True)

# Process all TIFF files in batch
for filename in os.listdir(input_folder):
    if filename.endswith('.tif'):
        input_file = os.path.join(input_folder, filename)  # Construct full path
        print(f"Processing file: {input_file}")
        
        with rasterio.open(input_file) as src:
            data = src.read(1)  # Read the first band
            profile = src.profile
            
            # Set new resolution (0.05) and calculate scale factor
            scale_factor = 5  # Aggregation factor from 0.01 to 0.05
            new_height = data.shape[0] // scale_factor
            new_width = data.shape[1] // scale_factor

            # Mean aggregation resampling
            resampled_data = data[:new_height * scale_factor, :new_width * scale_factor].reshape(
                new_height, scale_factor, new_width, scale_factor).mean(axis=(1, 3))

            # Update profile
            profile.update({
                "height": new_height,
                "width": new_width,
                "transform": src.transform * src.transform.scale(scale_factor, scale_factor)
            })

        # Generate new output filename
        new_filename = filename.replace("0.01", "0.05")  # Replace "0.01" in the filename with "0.05"
        output_file = os.path.join(output_folder, new_filename)

        # Export the result
        with rasterio.open(output_file, "w", **profile) as dst:
            dst.write(resampled_data, 1)

        print(f"File saved to: {output_file}")

print("All files have been processed.")

# End of the script